In [ ]:
file_path = '/home/andrey/PycharmProjects/MS-InsuranceScoring/US_Accidents_June20.csv'
# https://smoosavi.org/datasets/us_accidents

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from hyperopt import hp

In [3]:
from insolver.InsolverDataFrame import InsolverDataFrame
from insolver.InsolverTransforms import (
    TransformExp,
    InsolverTransformMain,
    InsolverTransforms,
    TransformAge,
    TransformMapValues,
    TransformPolynomizer,
    TransformAgeGender,
)
from insolver.InsolverWrapperGBM import InsolverGradientBoostingWrapper
from insolver.InsolverPlotsGBM import PredictionPlots, ShapPlots
from insolver.InsolverWrapperGLM import InsolverGLMWrapper
from insolver.InsolverUtils import train_val_test_split

In [4]:
df = pd.read_csv(file_path, low_memory=False)
df.shape

(3513740, 49)

In [5]:
df = df.sample(int(3513740/100))

In [6]:
df = df.drop(columns=['ID', 'Source', 'End_Lat', 'End_Lng'])
df.dropna(how='all', axis=1, inplace=True)

In [7]:
InsDataFrame = InsolverDataFrame(df)
InsDataFrame.head()

,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,Street,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
1211366,201.0,2,2019-04-03 07:47:08,2019-04-03 08:46:56,37.504467,-77.084549,0.00,Accident on VA-106 Emmaus Church Rd Northbound...,NaN,I-64 W,...,False,False,False,False,False,False,Day,Day,Day,Day
761314,201.0,3,2019-12-04 18:21:04,2019-12-04 19:20:50,37.467751,-122.291389,0.00,#1 lane blocked due to accident on I-280 North...,NaN,I-280 N,...,False,False,False,False,False,False,Night,Night,Night,Day
3061096,NaN,2,2019-12-20 12:05:00,2019-12-20 13:18:03,34.057322,-117.958810,0.00,At S Orange Ave - Accident.,1698.0,S Orange Ave,...,False,True,False,False,True,False,Day,Day,Day,Day
3168122,NaN,4,2019-03-20 00:18:56,2019-03-20 00:46:23,33.821950,-118.206850,0.08,Ramp to I-405 and Pacific Pl/Warnock Way - Acc...,NaN,Long Beach Fwy N,...,False,False,False,False,False,False,Night,Night,Night,Night
2079955,201.0,3,2018-02-15 05:40:18,2018-02-15 06:39:31,30.640692,-84.970779,0.00,Right hand shoulder blocked due to accident on...,NaN,FL-8 W,...,False,False,False,False,False,False,Night,Night,Day,Day


In [8]:
iglm = InsolverGLMWrapper()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /home/andrey/PycharmProjects/InsolverPackageExample/venv/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpxt8abal1
  JVM stdout: /tmp/tmpxt8abal1/h2o_andrey_started_from_python.out
  JVM stderr: /tmp/tmpxt8abal1/h2o_andrey_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_andrey_i0asht
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.844 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [9]:
InsTransforms = InsolverTransforms(InsDataFrame.get_data(), [])
InsTransforms.transform()

{}

In [10]:
df = InsTransforms.get_data()

In [11]:
train, valid, test = InsTransforms.split_frame(val_size=0.15, test_size=0.15, random_state=0, shuffle=True)

In [12]:
features = [
    'Start_Time',
    'End_Time',
    'Start_Lat',
    'Start_Lng',
    'Distance(mi)',
#     'Number',
#     'Street',
#     'Side',
#     'City',
#     'County',
#     'State',
#     'Zipcode',
#     'Country',
#     'Timezone',
#     'Airport_Code',
    'Weather_Timestamp',
    'Temperature(F)',
#     'Wind_Chill(F)',
#     'Humidity(%)',
#     'Pressure(in)',
#     'Visibility(mi)',
#     'Wind_Direction',
#     'Wind_Speed(mph)',
#     'Precipitation(in)',
#     'Weather_Condition',
#     'Amenity',
#     'Bump',
#     'Crossing',
#     'Give_Way',
#     'Junction',
#     'No_Exit',
#     'Railway',
#     'Roundabout',
#     'Station',
#     'Stop',
#     'Traffic_Calming',
#     'Traffic_Signal',
#     'Turning_Loop',
#     'Sunrise_Sunset',
#     'Civil_Twilight',
#     'Nautical_Twilight',
    'Astronomical_Twilight'
]
target = 'Severity'

In [13]:
iglm.model_init(train, valid, family='gamma', link='log')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
params = {'lambda': [1, 0.5, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0],
          'alpha': [i * 0.1 for i in range(0, 11)]}

In [15]:
iglm.grid_search_cv(features, target, params, search_criteria={'strategy': "Cartesian"})

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [16]:
iglm.save_model('glm')

In [17]:
predict_glm = iglm.predict(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [18]:
predict_glm

array([[2.29385169],
       [2.19173537],
       [2.48754102],
       ...,
       [2.20111302],
       [2.3534096 ],
       [2.33758487]])